# 1) Preparación previa

### <u>Carga de librerías</u>

In [ ]:
%%capture [--no-stderr]

import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import random

from pyproj import Geod
from scipy.stats import pearsonr
from scipy.stats import spearmanr

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### <u>Carga de datasets</u>

In [ ]:
dataset = pd.read_csv("dataset.csv", low_memory = False)
dataset

In [ ]:
dataset_usuarios_2020 = pd.read_csv("../Datasets/usuarios_ecobici_2020.csv")
dataset_usuarios_2019 = pd.read_csv("../Datasets/usuarios_ecobici_2019.csv")
dataset_usuarios_2018 = pd.read_csv("../Datasets/usuarios-ecobici-2018.csv").rename(columns={'usuario_id': 'id_usuario', 'usuario_sexo': 'genero_usuario', 'usuario_edad': 'edad_usuario'})
dataset_usuarios_2017 = pd.read_csv("../Datasets/usuarios-ecobici-2017.csv").rename(columns={'usuario_id': 'id_usuario', 'usuario_sexo': 'genero_usuario', 'usuario_edad': 'edad_usuario'})
dataset_usuarios_2016 = pd.read_csv("../Datasets/usuarios-ecobici-2016.csv").rename(columns={'usuario_id': 'id_usuario', 'usuario_sexo': 'genero_usuario', 'usuario_edad': 'edad_usuario'})
dataset_usuarios_2015 = pd.read_csv("../Datasets/usuarios-ecobici-2015.csv").rename(columns={'usuario_id': 'id_usuario', 'usuario_sexo': 'genero_usuario', 'usuario_edad': 'edad_usuario'})
dataset_usuarios = pd.concat([dataset_usuarios_2015, dataset_usuarios_2016, dataset_usuarios_2017, dataset_usuarios_2018, dataset_usuarios_2019, dataset_usuarios_2020])
dataset_usuarios

In [ ]:
dataset_usuarios.isnull().sum()

In [ ]:
dataset_usuarios.dropna(inplace = True)
dataset_usuarios

# 2) Gráficos

## <u>1er gráfico: análisis de género</u>

En primer lugar, se procede a definir los datos a utilizar, agrupando los totales según la variable que corresponda.

#### A) Recorridos según género

In [ ]:
round(dataset.genero_usuario.value_counts(normalize = True)*100,3)

Debido a la pequeña propoción de la respuesta N y a que solo se encuentra en 2 de los 7 años, se dejará de lado en el gráfico.

In [ ]:
dataset_genero = dataset.loc[dataset["genero_usuario"] != "N",:]
dataset_genero

In [ ]:
dataset_genero = dataset_genero.groupby("año").genero_usuario
totales_x_año_genero = dataset_genero.value_counts()
totales_x_año_genero

In [ ]:
porcentaje_x_año_genero = dataset_genero.value_counts(normalize = True).round(3)*100
porcentaje_x_año_genero

In [ ]:
totales_x_año_genero_rename = totales_x_año_genero.rename_axis(["año","genero"]).reset_index(name = "totales")
porcentaje_x_año_genero_rename = porcentaje_x_año_genero.rename_axis(["año","genero"]).reset_index(name = "porcentaje")

df_genero_x_año = pd.merge(totales_x_año_genero_rename, porcentaje_x_año_genero_rename, how = "outer")
df_genero_x_año

In [ ]:
totales_gpa_m = df_genero_x_año[df_genero_x_año["genero"] == "M"]
totales_gpa_f = df_genero_x_año[df_genero_x_año["genero"] == "F"]

#### B) Usuarios según género

In [ ]:
dataset_usuarios_genero = dataset.dropna(subset = ["id_usuario"]).drop_duplicates(subset = ["id_usuario"])
dataset_usuarios_genero

In [ ]:
dataset_usuarios_genero = dataset_usuarios_genero.genero_usuario.value_counts().rename_axis('genero').reset_index(name='totales')
dataset_usuarios_genero

In [ ]:
dataset_usuarios_genero = dataset_usuarios_genero.loc[dataset_usuarios_genero.genero != "N",:]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20,10), gridspec_kw={'width_ratios': [3, 1]})

# Se define la paleta de colores a utilizar:
colors = {'M':'coral', 'F':'aquamarine'} 

# Axis 1 = recorridos según género
#ax1.plot(totales_gpa_m["año"], totales_gpa_m["totales"], color='black', linewidth=2, label = "Total")
#ax1.plot(totales_gpa_f["año"], totales_gpa_f["totales"], color='black', linewidth=2, label = "Total")
width = 0.4
ax1.bar(totales_gpa_m["año"], totales_gpa_m["totales"], -width, color= colors["M"], align = "edge")
ax1.bar(totales_gpa_m["año"], totales_gpa_f["totales"], width, color= colors["F"], align = "edge")

# Propiedades
ax1.set_axisbelow(True)
ax1.set_title('Recorridos anuales según género', fontsize=14)
ax1.set_ylabel('Total en millones', fontsize=14)
ax1.set_xlabel('Año', fontsize=14)
ax1.grid(which='major', axis='y', color='black', lw=0.4, alpha=0.6)
ax1.grid(which='major', axis='x', color='black', lw=0.4, alpha=0.6)        

# Datos de barras
for p in ax1.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    ax1.annotate(f'{height}', (x + width/2, y + height+30000), ha='center', fontsize = 9)
    ax1.annotate(f'{round(100/df_genero_x_año.loc[df_genero_x_año.año == x, "totales"].sum()*height)}%', (x + width/2, y + height*0.4), ha='center', fontsize = 9)
    
# Leyenda
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
legend = ax1.legend(handles, labels, fontsize = 10)



# Axis 2 = total de usuarios según género
plt.sca(ax2) 
dataset_usuarios_genero.plot(ax = ax2, kind = "bar", width = 0.5, color = ["coral", "aquamarine"], x="genero", y="totales", rot = 0)

# Propiedades
ax2.set_axisbelow(True)
ax2.set_title('Usuarios según género', fontsize=14)
ax2.set_xlabel("Género", fontsize=14)
ax2.set_ylabel("Total", fontsize=14)
ax2.grid(which='major', axis='y', color='black', lw=0.4, alpha=0.6)

# Datos de barras
for p in ax2.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    ax2.annotate(f'{height}', (x + width/2, y + height + 3000), ha='center')
    ax2.annotate(f'{round(100/dataset_usuarios_genero.totales.sum()*height)}%', (x + width/2, y + height*0.5), ha='center')

# Leyenda
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
legend = ax2.legend(handles, labels, fontsize = 10)

plt.savefig("Análisis de género.jpg", dpi = 300)

## <u>2do gráfico: análisis de duración y distancia</u>

In [ ]:
wgs84_geod = Geod(ellps='WGS84')

def distancia(lat1, lon1, lat2, lon2):
  az12, az21, dist = wgs84_geod.inv(lon1,lat1,lon2,lat2)
  return dist

dataset['distancia'] = distancia(dataset['lat_estacion_origen'].tolist(), dataset['long_estacion_origen'].tolist(), dataset['lat_estacion_destino'].tolist(), dataset['long_estacion_destino'].tolist())

In [ ]:
dataset

In [ ]:
q75,q25 = np.percentile(dataset["distancia"],[75,25])
iqr = q75-q25
max_limit = q75+(1.5*iqr)
min_limit = q25-(1.5*iqr)
print("límite superior:", max_limit, "\nlímite inferior:", min_limit)

In [ ]:
dataset_distancia = dataset.loc[(dataset["distancia"] < max_limit) & (dataset["distancia"]  > min_limit), ["año", "distancia"]]

print("Se borraron", len(dataset) - len(dataset_distancia), "outliers, quedando un total de", len(dataset_distancia), "valores válidos entre 0 y 60 minutos")

In [ ]:
fig = plt.figure(figsize = (20,20))

# Axis 1 = distribución anual de la distancia de los recorridos
ax1 = plt.subplot(2, 1, 1)
sns.violinplot(data = dataset_distancia, x = "año", y = "distancia")
ax1.set_axisbelow(True)
ax1.yaxis.grid(which='major', color='black', lw=0.4, alpha=0.6, linestyle='-')    
plt.title('Distribución anual de la distancia de los recorridos', fontsize=14)
plt.ylabel('Distancia en metros', fontsize=14)
plt.xlabel('Año', fontsize=14)

# Axis 2 = distribución anual de la duración de los recorridos
ax2 = plt.subplot(2, 1, 2)
sns.violinplot(data = dataset, x = "año", y = "minutos")
ax2.set_axisbelow(True)
ax2.yaxis.grid(which='major', color='black', lw=0.4, alpha=0.6, linestyle='-')
plt.title('Distribución anual de la duración de los recorridos', fontsize=14)
plt.ylabel('Duración en minutos', fontsize=14)
plt.xlabel('Año', fontsize=14)

plt.savefig("Análisis de distancia y duración.jpg", dpi = 300)

## <u>3er gráfico: análisis de tipo de viaje</u>

In [ ]:
dataset["misma_estacion"] = dataset.codigo_origen == dataset.codigo_destino
df_misma_estacion = dataset["misma_estacion"].value_counts().rename_axis(["valor"]).reset_index(name = "totales")
df_misma_estacion

In [ ]:
plt.figure(facecolor = 'white', figsize=(10, 10))
patches, texts = plt.pie(df_misma_estacion["totales"],
                         labels = [str(round(dataset["misma_estacion"].value_counts(normalize = True)[0]*100,2)) + "%", str(round(dataset["misma_estacion"].value_counts(normalize = True)[1]*100,2)) + "%"],
                         labeldistance = 0.5)
texts[0].set_fontsize(12)
texts[1].set_fontsize(12)
plt.title('Proporción de recorridos que finalizan en la estación de origen', fontsize=18)
plt.legend(["Sí", "No"], fontsize = 14)

plt.savefig("Análisis de tipo de viaje.jpg", dpi = 300)

## <u>4to gráfico: análisis de tipo de estaciones</u>

In [ ]:
estaciones_requeridas_origen_porcentaje = round((dataset.loc[:,["codigo_origen"]].codigo_origen.value_counts(normalize = True)*100)[0:41],2)
estaciones_requeridas_origen_porcentaje = pd.DataFrame(estaciones_requeridas_origen_porcentaje.rename_axis("codigo_origen").reset_index(name = "porcentaje_recorridos"))
df_codigos_nombres_origen = dataset.loc[:,["codigo_origen", "nombre_origen"]]
df_codigos_nombres_origen.drop_duplicates(subset = ["codigo_origen"], inplace = True)
estaciones_requeridas_origen_porcentaje = estaciones_requeridas_origen_porcentaje.merge(df_codigos_nombres_origen, on = "codigo_origen", how = "left")
estaciones_requeridas_origen_porcentaje

In [ ]:
round(estaciones_requeridas_origen_porcentaje.porcentaje_recorridos.sum(),2)

In [ ]:
estaciones_requeridas_destino_porcentaje = round((dataset.loc[:,["codigo_destino"]].codigo_destino.value_counts(normalize = True)*100)[0:41],2)
estaciones_requeridas_destino_porcentaje = pd.DataFrame(estaciones_requeridas_destino_porcentaje.rename_axis("codigo_destino").reset_index(name = "porcentaje_recorridos"))
df_codigos_nombres_destino = dataset.loc[:,["codigo_destino", "nombre_destino"]]
df_codigos_nombres_destino.drop_duplicates(subset = ["codigo_destino"], inplace = True)
estaciones_requeridas_destino_porcentaje = estaciones_requeridas_destino_porcentaje.merge(df_codigos_nombres_destino, on = "codigo_destino", how = "left")
estaciones_requeridas_destino_porcentaje

In [ ]:
round(estaciones_requeridas_destino_porcentaje.porcentaje_recorridos.sum(),2)

In [ ]:
estaciones_requeridas_origen = dataset.loc[:,["codigo_origen"]].codigo_origen.value_counts()[0:41]
estaciones_requeridas_origen = pd.DataFrame(estaciones_requeridas_origen.rename_axis("codigo_origen").reset_index(name = "cantidad_recorridos_origen"))
df_codigos_nombres_origen = dataset.loc[:,["codigo_origen", "nombre_origen"]]
df_codigos_nombres_origen.drop_duplicates(subset = ["codigo_origen"], inplace = True)
estaciones_requeridas_origen = estaciones_requeridas_origen.merge(df_codigos_nombres_origen, on = "codigo_origen", how = "left")
estaciones_requeridas_origen.rename(columns = {"codigo_origen" : "codigo"}, inplace = True)
estaciones_requeridas_origen

In [ ]:
estaciones_requeridas_destino = dataset.loc[:,["codigo_destino"]].codigo_destino.value_counts()[0:41]
estaciones_requeridas_destino = pd.DataFrame(estaciones_requeridas_destino.rename_axis("codigo_destino").reset_index(name = "cantidad_recorridos_destino"))
df_codigos_nombres_destino = dataset.loc[:,["codigo_destino", "nombre_destino"]]
df_codigos_nombres_destino.drop_duplicates(subset = ["codigo_destino"], inplace = True)
estaciones_requeridas_destino = estaciones_requeridas_destino.merge(df_codigos_nombres_destino, on = "codigo_destino", how = "left")
estaciones_requeridas_destino.rename(columns = {"codigo_destino" : "codigo"}, inplace = True)
estaciones_requeridas_destino

In [ ]:
estaciones_requeridas = estaciones_requeridas_origen.merge(estaciones_requeridas_destino, on = "codigo", how = "left")
estaciones_requeridas.dropna(subset = ["cantidad_recorridos_destino", "nombre_destino"], inplace = True)
estaciones_requeridas

In [ ]:
# Cálculo de correlación con Pearson y Spearman para utilizar en el gráfico
corr, _ = pearsonr(estaciones_requeridas["cantidad_recorridos_origen"], estaciones_requeridas["cantidad_recorridos_destino"])
corr2, _ = spearmanr(estaciones_requeridas["cantidad_recorridos_origen"], estaciones_requeridas["cantidad_recorridos_destino"])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (17,10), gridspec_kw={'width_ratios': [2.5, 0.8]})

# Creación de listado randomizado de colores
color_estaciones = []
for i in range(len(estaciones_requeridas)):
    color_estaciones.append('#%06X' % random.randint(0, 0xFFFFFF))

# Axis 1 = recorridos originados y finalizados en cada estación
ax1.scatter(estaciones_requeridas["cantidad_recorridos_origen"], estaciones_requeridas["cantidad_recorridos_destino"], color = color_estaciones)
ax1.set_xlabel("Cantidad de recorridos que partieron de la estación", fontsize = 12)
ax1.set_ylabel("Cantidad de recorridos que finalizaron en la estación", fontsize = 12)
ax1.set_title('Recorridos originados y finalizados en cada estación', fontsize=18)
ax1.grid(which='major', axis='y', color='black', lw=0.4, alpha=0.6)
ax1.grid(which='major', axis='x', color='black', lw=0.4, alpha=0.6)
ax1.set_axisbelow(True)
ax1.set_xlim(left = 60000, right = 220000)
ax1.set_ylim(bottom = 60000, top = 200000)
for i, txt in enumerate(estaciones_requeridas.loc[0:8, "nombre_origen"]):
    ax1.annotate(txt, (estaciones_requeridas.cantidad_recorridos_origen[i]+1200, estaciones_requeridas.cantidad_recorridos_destino[i]-600))



# Axis 2 = tests de correlación
ax2.set_title('Test de correlación', fontsize=18, x = 0.3)
ax2.set_axis_off()
ax2.text(0.06, 0.75, f'Pearson:{round(corr,3)}', fontsize = 14)
ax2.text(0.02, 0.65, f'Spearman:{round(corr2,3)}', fontsize = 14)
ax2.text(0.32, 0.3, 'Hay una alta correlación\nentre la variable origen\ny la variable destino\nde los recorridos', horizontalalignment='center', fontsize = 14)

plt.savefig("Análisis de correlación entre origen y destino.jpg", dpi = 300)

## <u>5to gráfico: análisis de usuario (edad)</u>

In [ ]:
dataset_usuarios

In [ ]:
q75,q25 = np.percentile(dataset_usuarios["edad_usuario"],[75,25])
iqr = q75-q25
max_limit = q75+(1.5*iqr)
min_limit = q25-(1.5*iqr)
print("límite superior:", max_limit, "\nlímite inferior:", min_limit)

In [ ]:
# El límite superior no se utilizará pues podría eliminar información importante, se reemplaza por uno más lógico
max_limit = 90
dataset_usuarios_edad = dataset_usuarios.loc[(dataset_usuarios["edad_usuario"] < max_limit) & (dataset_usuarios["edad_usuario"]  > min_limit), :]
print("Se borraron", len(dataset_usuarios) - len(dataset_usuarios_edad), "outliers, quedando un total de", len(dataset_usuarios_edad), "valores válidos de edades entre 0 y 90 años")

In [ ]:
bin_list = np.arange(10, 100, 2.5)
plt.figure(figsize=(18, 10))
n, bins, patches = plt.hist(x = dataset_usuarios_edad.edad_usuario, bins = bin_list, color='mediumseagreen', rwidth=0.85)
maxfreq = n.max()
plt.rc('axes', axisbelow=True)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Edad', fontsize = 14)
plt.ylabel('Cantidad de usuarios', fontsize = 14)
plt.title('Usuarios según edad', fontsize = 18)
plt.ylim(ymax = 38000)
plt.xlim(xmax = 90)
plt.xticks(np.arange(0, 91, 5))

plt.savefig("Análisis de usuarios (edad).jpg", dpi = 300)

## <u>6to gráfico: análisis de usuario (cantidad de recorridos)</u>

In [ ]:
dataset

In [ ]:
dataset_usuarios_recorridos = dataset.dropna(subset = ["id_usuario"])
dataset_usuarios_recorridos = dataset_usuarios_recorridos.id_usuario.value_counts().rename_axis('id_usuario').reset_index(name='totales')
dataset_usuarios_recorridos

In [ ]:
dataset_usuarios_recorridos.totales.value_counts()

In [ ]:
q75,q25 = np.percentile(dataset_usuarios_recorridos["totales"],[75,25])
iqr = q75-q25
max_limit = q75+(1.5*iqr)
min_limit = q25-(1.5*iqr)
print("límite superior:", max_limit, "\nlímite inferior:", min_limit)

In [ ]:
# El límite superior no se utilizará pues podría eliminar información importante, se reemplaza por uno más lógico
max_limit = 1000
dataset_usuarios_recorridos_sin_outliers = dataset_usuarios_recorridos.loc[(dataset_usuarios_recorridos["totales"] < max_limit) & (dataset_usuarios_recorridos["totales"]  > min_limit), :]
print("Se borraron", len(dataset_usuarios_recorridos) - len(dataset_usuarios_recorridos_sin_outliers), "outliers, quedando un total de", len(dataset_usuarios_recorridos_sin_outliers), "usuarios que viajaron entre 0 y 1000 de edades entre 0 y 90 años")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (20,20), gridspec_kw={'height_ratios': [1, 2]})

# Axis 1 = usuarios según cantidad de recorridos
plt.sca(ax1)
bin_list = np.arange(0, 300, 20)
ax1, bins, patches = plt.hist(x = dataset_usuarios_recorridos_sin_outliers.totales, bins = bin_list, color='mediumturquoise', rwidth=0.85, orientation="horizontal")
maxfreq = ax1.max()
plt.rc('axes', axisbelow=True)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Cantidad de usuarios', fontsize = 14)
plt.ylabel('Cantidad de recorridos', fontsize = 14)
plt.title('Usuarios según cantidad de recorridos', fontsize = 18)
plt.yticks(np.arange(0, 301, 100))

# Axis 2 = usuarios según cantidad de viajes con escala logarítmica
plt.sca(ax2)
bin_list = np.arange(0, 1000, 20)
ax2, bins, patches = plt.hist(x = dataset_usuarios_recorridos_sin_outliers.totales, bins = bin_list, color='mediumturquoise', rwidth=0.85)
maxfreq = ax2.max()
plt.rc('axes', axisbelow=True)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Cantidad de recorridos', fontsize = 14)
plt.ylabel('Cantidad de usuarios en escala logarítmica', fontsize = 14)
plt.title('Usuarios según cantidad de recorridos en escala logarítmica', fontsize = 18)
plt.yscale("log")
plt.xticks(np.arange(0, 1001, 100))

plt.savefig("Análisis de usuarios (recorridos).jpg", dpi = 300)